# * VINSIGHT : Gross Adds
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales, AE

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000101CS' --Prepaid Gross Adds
        , 'TB1S000101CS' --Prepaid Gross Adds : TMH
        , 'DB1S000101CS' --Prepaid Gross Adds : DTAC
        , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
        , 'TB1S000101CG' --Prepaid Gross Adds : TMH - GEO Channel
        , 'DB1S000101CG' --Prepaid Gross Adds : DTAC - GEO Channel
        
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB2S010100CS' --Postpaid Gross Adds B2C : TMH
        , 'DB2S010100CS' --Postpaid Gross Adds B2C : DTAC
        , 'B2S020100CS' --Postpaid Gross Adds B2B
        , 'TB2S020100CS' --Postpaid Gross Adds B2B : TMH
        , 'DB2S020100CS' --Postpaid Gross Adds B2B : DTAC
        , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
        , 'TB2S010100CG' --Postpaid Gross Adds B2C : TMH - GEO Channel
        , 'DB2S010100CG' --Postpaid Gross Adds B2C : DTAC - GEO Channel
        , 'B2S020100CG' --Postpaid Gross Adds B2B - GEO Channel
        , 'TB2S020100CG' --Postpaid Gross Adds B2B : TMH - GEO Channel
        , 'DB2S020100CG' --Postpaid Gross Adds B2B : DTAC - GEO Channel
        
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
        , 'TB3S000102D1CS' --TOL Gross Adds Connected : Consumer (Install Location)
        , 'TB3S000102D1CG' --TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
        , 'TB3S000103CS' --TOL Gross Adds Connected : DataService
        , 'TB3S000103CG' --TOL Gross Adds Connected : DataService - GEO Channel
        , 'TB3S000103D1CS' --TOL Gross Adds Connected : DataService (Install Location)
        , 'TB3S000103D1CG' --TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
        , 'TB3S000104CS' --TOL Gross Adds Registration : Consumer
        , 'TB3S000104CG' --TOL Gross Adds Registration : Consumer - GEO Channel
        , 'TB3S000105CS' --TOL Gross Adds Registration : DataService
        , 'TB3S000105CG' --TOL Gross Adds Registration : DataService - GEO Channel
        
        , 'TB4S001004CS' --TVS CMDU Gross Adds
        , 'TB4S001004CG' --TVS CMDU Gross Adds - GEO Channel
        , 'TB4S001400CS' --TVS Now Gross Adds
        , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-07-03, 11:41:41

DataFrame: 6222 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_22264\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Gross Adds(All Channel) Yearly '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['PRE_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['PRE_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB2S010100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B'] = np.where(ga_yearly_df['METRIC_CD']=='B2S020100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB2S020100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB2S020100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_REG_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000104CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_CONN'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_CONN(Install)'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102D1CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_REG_DSV'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000105CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_DSV'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000103CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_DSV(Install)'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000103D1CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TVS_CMDU'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001004CS', ga_yearly_df['P'], 0)

# ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_yearly_df_display = ga_yearly_df.copy()
ga_yearly_df_display['ACTUAL_AS_OF'] = ga_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df_display[col] = ga_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025,2025-07-03 03:13:24,20250701,"12,175,482","5,818,892","6,395,810","1,119,610","856,107","268,755","410,086","298,322","9,635"


In [5]:
''' Gross Adds(All Channel) Quarterly '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]

ga_quarterly_df['PRE'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['PRE_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['PRE_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB2S010100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S020100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB2S020100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB2S020100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_REG_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000104CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_CONN'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_CONN(Install)'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102D1CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_REG_DSV'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000105CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_DSV'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000103CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_DSV(Install)'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000103D1CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TVS_CMDU'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001004CS', ga_quarterly_df['P'], 0)

# ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_quarterly_df_display = ga_quarterly_df.copy()
ga_quarterly_df_display['ACTUAL_AS_OF'] = ga_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df_display[col] = ga_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,20251,2025-07-03 03:13:24,20250331,"6,673,218","3,141,270","3,531,948","569,340","438,816","130,524","198,847","145,692","3,337"
1,20252,2025-07-03 03:13:24,20250630,"5,502,264","2,677,622","2,863,862","550,270","412,039","138,231","208,987","150,911","6,298"
2,20253,2025-07-03 03:13:24,20250701,0,0,0,0,"5,252",0,"2,252","1,719",0


In [6]:
''' Gross Adds(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]

ga_monthly_df['PRE'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['PRE_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['PRE_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB2S010100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B'] = np.where(ga_monthly_df['METRIC_CD']=='B2S020100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB2S020100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB2S020100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_REG_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000104CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_CONN'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_CONN(Install)'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102D1CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_REG_DSV'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000105CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_DSV'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000103CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_DSV(Install)'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000103D1CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TVS_CMDU'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001004CS', ga_monthly_df['P'], 0)

# ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_monthly_df_display = ga_monthly_df.copy()
ga_monthly_df_display['ACTUAL_AS_OF'] = ga_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df_display[col] = ga_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202501,2025-07-03 03:13:24,20250131,"2,507,456","1,166,199","1,341,257","188,248","142,922","45,326","64,026","46,939",769
1,202502,2025-07-03 03:13:24,20250228,"2,161,914","999,208","1,162,706","183,973","140,856","43,117","62,508","45,846",545
2,202503,2025-07-03 03:13:24,20250331,"2,003,848","975,863","1,027,985","197,119","155,038","42,081","72,313","52,907","2,023"
3,202504,2025-07-03 03:13:24,20250430,"1,879,322","931,089","948,233","197,270","152,200","45,070","66,846","48,482","2,352"
4,202505,2025-07-03 03:13:24,20250531,"1,912,819","912,219","1,000,600","182,723","134,956","47,767","72,984","52,741","2,206"
5,202506,2025-07-03 03:13:24,20250630,"1,710,123","834,314","915,029","170,277","124,883","45,394","69,157","49,688","1,740"
6,202507,2025-07-03 03:13:24,20250701,0,0,0,0,"5,252",0,"2,252","1,719",0


In [7]:
''' Gross Adds(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]

ga_weekly_df['PRE'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['PRE_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['PRE_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB2S010100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B'] = np.where(ga_weekly_df['METRIC_CD']=='B2S020100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB2S020100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB2S020100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_REG_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000104CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_CONN'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_CONN(Install)'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102D1CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_REG_DSV'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000105CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_DSV'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000103CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_DSV(Install)'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000103D1CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TVS_CMDU'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001004CS', ga_weekly_df['P'], 0)

# ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_weekly_df_display = ga_weekly_df.copy()
ga_weekly_df_display['ACTUAL_AS_OF'] = ga_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df_display[col] = ga_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025022,2025-07-03 03:13:24,20250601,"64,643","28,258","36,385","5,571","4,603",968,"2,026","1,670",69
1,2025023,2025-07-03 03:13:24,20250608,"421,599","192,353","229,246","42,351","32,135","10,216","17,093","12,378",341
2,2025024,2025-07-03 03:13:24,20250615,"387,136","183,705","203,431","40,541","30,697","9,844","16,483","11,591",281
3,2025025,2025-07-03 03:13:24,20250622,"396,623","185,078","211,545","39,736","27,223","12,513","16,133","11,969",358
4,2025026,2025-07-03 03:13:24,20250629,"440,122","205,700","234,422","36,114","26,117","9,997","17,422","12,080",508
5,2025027,2025-07-03 03:13:24,20250701,0,"39,220",0,"5,964","9,360","1,856","2,252","1,719",183


In [8]:
''' Gross Adds(All Channel) Daily '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]

ga_daily_df['PRE'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_T'] = np.where(ga_daily_df['METRIC_CD']=='TB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_D'] = np.where(ga_daily_df['METRIC_CD']=='DB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S010100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_REG_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000104CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_CONN'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_CONN(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102D1CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_REG_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000105CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103D1CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)
# ga_daily_df['TVS_CMDU'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001004CS', ga_daily_df['P'], 0)

# ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_daily_df_display = ga_daily_df.copy()
mod_col_list = ga_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df_display[col] = ga_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202507,20250702,2025-07-03 03:13:24,0,0,0,0,0,0,0,0,0
1,202507,20250701,2025-07-03 03:13:24,0,0,0,0,"5,252",0,"2,252","1,719",0
2,202506,20250630,2025-07-03 03:13:24,0,"39,220",0,"5,964","4,108","1,856",0,0,183
3,202506,20250629,2025-07-03 03:13:24,"61,926","28,248","33,678","5,231","3,812","1,419","2,265","1,816",97
4,202506,20250628,2025-07-03 03:13:24,"66,513","32,087","34,426","6,151","4,824","1,327","2,522","2,001",91
5,202506,20250627,2025-07-03 03:13:24,"73,911","35,501","38,410","4,710","3,347","1,363","2,551","1,864",73
6,202506,20250626,2025-07-03 03:13:24,"67,875","34,683","33,192","4,481","3,179","1,302","2,901","1,634",57
7,202506,20250625,2025-07-03 03:13:24,"58,617","22,490","36,127","4,806","3,156","1,650","2,082","1,567",66
8,202506,20250624,2025-07-03 03:13:24,"56,195","26,652","29,543","5,742","4,283","1,459","2,663","1,581",63
9,202506,20250623,2025-07-03 03:13:24,"55,085","26,039","29,046","4,993","3,516","1,477","2,438","1,617",61


## Overview(Geo) by Period

In [9]:
''' Gross Adds(Geo Channel) Yearly '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_geo_yearly_df = ga_geo_yearly_df.loc[ga_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_yearly_df['PRE'] = np.where(ga_geo_yearly_df['METRIC_CD']=='B1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['PRE_T'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['PRE_D'] = np.where(ga_geo_yearly_df['METRIC_CD']=='DB1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C'] = np.where(ga_geo_yearly_df['METRIC_CD']=='B2S010100CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C_T'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB2S010100CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C_D'] = np.where(ga_geo_yearly_df['METRIC_CD']=='DB2S010100CG', ga_geo_yearly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_yearly_df['TOL_REG_CON'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000104CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['TOL_CONN'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000102CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_CONN(Install)'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_REG_DSV'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000105CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_DSV'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000103CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_DSV(Install)'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['TVS_NOW'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB4S001400CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TVS_CMDU'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB4S001004CG', ga_geo_yearly_df['P'], 0)

# ga_geo_yearly_df = ga_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_yearly_df = ga_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# ga_geo_yearly_df = ga_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_yearly_df = ga_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_yearly_df = ga_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_geo_yearly_df = ga_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_yearly_df_display = ga_geo_yearly_df.copy()
ga_geo_yearly_df_display['ACTUAL_AS_OF'] = ga_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_yearly_df_display[col] = ga_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025,2025-07-03 03:13:24,20250701,"10,320,493","5,798,795","4,560,860","503,313","426,285","79,481","313,359","231,310","9,610"


In [10]:
''' Gross Adds(Geo Channel) Quarterly '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_geo_quarterly_df = ga_geo_quarterly_df.loc[ga_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_quarterly_df['PRE'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='B1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['PRE_T'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['PRE_D'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='DB1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='B2S010100CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C_T'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB2S010100CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C_D'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='DB2S010100CG', ga_geo_quarterly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_quarterly_df['TOL_REG_CON'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000104CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['TOL_CONN'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000102CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_CONN(Install)'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_REG_DSV'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000105CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_DSV'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000103CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_DSV(Install)'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['TVS_NOW'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB4S001400CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TVS_CMDU'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB4S001004CG', ga_geo_quarterly_df['P'], 0)

# ga_geo_quarterly_df = ga_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_quarterly_df = ga_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# ga_geo_quarterly_df = ga_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_quarterly_df = ga_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_quarterly_df = ga_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_geo_quarterly_df = ga_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_quarterly_df_display = ga_geo_quarterly_df.copy()
ga_geo_quarterly_df_display['ACTUAL_AS_OF'] = ga_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_quarterly_df_display[col] = ga_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,20251,2025-07-03 03:13:24,20250331,"5,553,809","3,132,044","2,421,765","262,488","223,283","39,205","150,337","111,869","3,329"
1,20252,2025-07-03 03:13:24,20250630,"4,766,684","2,666,751","2,139,095","240,825","200,549","40,276","161,268","118,044","6,281"
2,20253,2025-07-03 03:13:24,20250701,0,0,0,0,"2,453",0,"1,754","1,397",0


In [11]:
''' Gross Adds(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
ga_geo_monthly_df = ga_geo_monthly_df.loc[ga_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_monthly_df['PRE'] = np.where(ga_geo_monthly_df['METRIC_CD']=='B1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['PRE_T'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['PRE_D'] = np.where(ga_geo_monthly_df['METRIC_CD']=='DB1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C'] = np.where(ga_geo_monthly_df['METRIC_CD']=='B2S010100CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C_T'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB2S010100CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C_D'] = np.where(ga_geo_monthly_df['METRIC_CD']=='DB2S010100CG', ga_geo_monthly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_monthly_df['TOL_REG_CON'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000104CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['TOL_CONN'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000102CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_CONN(Install)'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_REG_DSV'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000105CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_DSV'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000103CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_DSV(Install)'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['TVS_NOW'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB4S001400CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TVS_CMDU'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB4S001004CG', ga_geo_monthly_df['P'], 0)

# ga_geo_monthly_df = ga_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_monthly_df = ga_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# ga_geo_monthly_df = ga_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_monthly_df = ga_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_monthly_df = ga_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_geo_monthly_df = ga_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_monthly_df_display = ga_geo_monthly_df.copy()
ga_geo_monthly_df_display['ACTUAL_AS_OF'] = ga_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_monthly_df_display[col] = ga_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202501,2025-07-03 03:13:24,20250131,"2,038,766","1,163,218","875,548","83,267","70,852","12,415","46,010","34,626",765
1,202502,2025-07-03 03:13:24,20250228,"1,798,194","996,201","801,993","83,956","71,425","12,531","48,150","35,785",543
2,202503,2025-07-03 03:13:24,20250331,"1,716,849","972,625","744,224","95,265","81,006","14,259","56,177","41,458","2,021"
3,202504,2025-07-03 03:13:24,20250430,"1,636,131","926,446","709,685","92,697","78,554","14,143","51,632","37,924","2,349"
4,202505,2025-07-03 03:13:24,20250531,"1,662,837","908,569","754,268","78,554","64,712","13,842","56,167","41,428","2,196"
5,202506,2025-07-03 03:13:24,20250630,"1,467,716","831,736","675,142","69,574","57,283","12,291","53,469","38,692","1,736"
6,202507,2025-07-03 03:13:24,20250701,0,0,0,0,"2,453",0,"1,754","1,397",0


In [12]:
''' Gross Adds(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_geo_weekly_df = ga_geo_weekly_df.loc[ga_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_weekly_df['PRE'] = np.where(ga_geo_weekly_df['METRIC_CD']=='B1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['PRE_T'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['PRE_D'] = np.where(ga_geo_weekly_df['METRIC_CD']=='DB1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C'] = np.where(ga_geo_weekly_df['METRIC_CD']=='B2S010100CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C_T'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB2S010100CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C_D'] = np.where(ga_geo_weekly_df['METRIC_CD']=='DB2S010100CG', ga_geo_weekly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_weekly_df['TOL_REG_CON'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000104CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['TOL_CONN'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000102CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_CONN(Install)'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_REG_DSV'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000105CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_DSV'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000103CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_DSV(Install)'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['TVS_NOW'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB4S001400CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TVS_CMDU'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB4S001004CG', ga_geo_weekly_df['P'], 0)

# ga_geo_weekly_df = ga_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_weekly_df = ga_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# ga_geo_weekly_df = ga_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_weekly_df = ga_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_weekly_df = ga_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_geo_weekly_df = ga_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_weekly_df_display = ga_geo_weekly_df.copy()
ga_geo_weekly_df_display['ACTUAL_AS_OF'] = ga_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_weekly_df_display[col] = ga_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025022,2025-07-03 03:13:24,20250601,"57,172","28,159","29,013","2,760","2,152",608,"1,604","1,240",69
1,2025023,2025-07-03 03:13:24,20250608,"367,486","191,795","175,691","17,547","14,339","3,208","13,251","9,631",340
2,2025024,2025-07-03 03:13:24,20250615,"332,223","182,931","149,292","16,293","13,613","2,680","12,671","9,103",279
3,2025025,2025-07-03 03:13:24,20250622,"336,601","184,487","152,114","15,543","12,903","2,640","12,602","9,381",358
4,2025026,2025-07-03 03:13:24,20250629,"374,234","205,202","169,032","14,888","12,186","2,702","13,341","9,337",508
5,2025027,2025-07-03 03:13:24,20250701,0,"39,162",0,"2,543","4,543",453,"1,754","1,397",182


In [13]:
''' Gross Adds(Geo Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_daily_df['PRE'] = np.where(ga_geo_daily_df['METRIC_CD']=='B1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C'] = np.where(ga_geo_daily_df['METRIC_CD']=='B2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB2S010100CG', ga_geo_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_daily_df['TOL_REG_CON'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000104CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TOL_CONN'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_CONN(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_REG_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000105CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TVS_NOW'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001400CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TVS_CMDU'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001004CG', ga_geo_daily_df['P'], 0)

# ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_daily_df_display = ga_geo_daily_df.copy()
mod_col_list = ga_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_daily_df_display[col] = ga_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202507,20250702,2025-07-03 03:13:24,0,0,0,0,0,0,0,0,0
1,202507,20250701,2025-07-03 03:13:24,0,0,0,0,"2,453",0,"1,754","1,397",0


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202507


### Prepaid

In [16]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Prepaid,B1S000101CS,Prepaid Gross Adds,2025-07-03 03:13:24,0,0,0,0,0
1,202507,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-07-03 03:13:24,0,0,0,0,0
2,202507,Prepaid,TB1S000101CS,Prepaid Gross Adds : TMH,2025-07-03 03:13:24,0,0,0,0,0


In [17]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Prepaid,B1S000101CG,Prepaid Gross Adds - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0
1,202507,Prepaid,DB1S000101CG,Prepaid Gross Adds : DTAC - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0
2,202507,Prepaid,TB1S000101CG,Prepaid Gross Adds : TMH - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0


### Postpaid

In [18]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2S010100CS,Postpaid Gross Adds B2C,2025-07-03 03:13:24,0,0,0,0,0
1,202507,Postpaid,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-07-03 03:13:24,0,0,0,0,0
2,202507,Postpaid,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-07-03 03:13:24,"5,252","5,252","3,856","3,856","3,856"


In [19]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2S010100CG,Postpaid Gross Adds B2C - GEO Channel,2025-07-03 03:13:24,0,0,0,0,73
1,202507,Postpaid,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0
2,202507,Postpaid,TB2S010100CG,Postpaid Gross Adds B2C : TMH - GEO Channel,2025-07-03 03:13:24,0,"2,453","2,453","2,453","2,453"


In [20]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2S020100CS,Postpaid Gross Adds B2B,2025-07-03 03:13:24,0,0,0,0,1
1,202507,Postpaid,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-07-03 03:13:24,0,0,0,0,0
2,202507,Postpaid,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-07-03 03:13:24,899,899,264,264,264


In [21]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,Postpaid,B2S020100CG,Postpaid Gross Adds B2B - GEO Channel,2025-07-03 03:13:24,0,0,0,17,23
1,202507,Postpaid,DB2S020100CG,Postpaid Gross Adds B2B : DTAC - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0
2,202507,Postpaid,TB2S020100CG,Postpaid Gross Adds B2B : TMH - GEO Channel,2025-07-03 03:13:24,0,264,264,264,264


### TOL

In [22]:
''' TOL Registration : Consumer (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Consumer')]
# tol_reg_con_df = tol_reg_con_df.loc[~tol_reg_con_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_con_df = tol_reg_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-07-03 03:13:24,"2,252","2,252","1,754","1,754","1,741"
1,202507,TOL,TB3S000104CG,TOL Gross Adds Registration : Consumer - GEO C...,2025-07-03 03:13:24,0,"1,754","1,754","1,754","1,741"


In [23]:
''' TOL Registration : DataService (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('DataService')]
# tol_reg_dsv_df = tol_reg_dsv_df.loc[~tol_reg_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_dsv_df = tol_reg_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-07-03 03:13:24,30,30,2,2,2
1,202507,TOL,TB3S000105CG,TOL Gross Adds Registration : DataService - GE...,2025-07-03 03:13:24,0,2,2,2,2


In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"1,719","1,719","1,398","1,398","1,378"
1,202507,TOL,TB3S000102D1CS,TOL Gross Adds Connected : Consumer (Install L...,2025-07-03 03:13:24,"1,719","1,719","1,719","1,719","1,719"


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3S000102CG,TOL Gross Adds Connected : Consumer - GEO Channel,2025-07-03 03:13:24,0,"1,397","1,397","1,397","1,377"
1,202507,TOL,TB3S000102D1CG,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-07-03 03:13:24,0,"1,397","1,397","1,397","1,397"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-07-03 03:13:24,55,55,4,4,4
1,202507,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-07-03 03:13:24,55,55,4,4,4


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-07-03 03:13:24,55,55,4,4,4
1,202507,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-07-03 03:13:24,55,55,4,4,4


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TVS,TB4S001400CS,TVS Now Gross Adds,2025-07-03 03:13:24,0,0,0,0,0
1,202507,TVS,TB4S001400CG,TVS Now Gross Adds - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202507,TVS,TB4S001004CS,TVS CMDU Gross Adds,2025-07-03 03:13:24,0,0,0,0,0
1,202507,TVS,TB4S001004CG,TVS CMDU Gross Adds - GEO Channel,2025-07-03 03:13:24,0,0,0,0,0


## ** Issue History

In [30]:
''' Monthly : TOL Gross Adds Connected : Consumer '''

v_metric_cd = 'TB3S000102CS'

issue_tol_ga_conn_con_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_tol_ga_conn_con_df = issue_tol_ga_conn_con_df.loc[issue_tol_ga_conn_con_df['TM_KEY_MTH']>=202401]
issue_tol_ga_conn_con_df = issue_tol_ga_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_tol_ga_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202501,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"46,939","46,939","34,630","34,630","36,115"
1,202502,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"45,846","45,846","35,794","35,319","35,506"
2,202503,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"52,907","52,907","41,460","41,460","41,080"
3,202504,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"48,482","48,482","37,922","37,883","37,550"
4,202505,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"52,741","52,741","41,427","41,427","41,039"
5,202506,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"49,688","49,688","38,690","39,652","39,515"
6,202507,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"1,719","1,719","1,398","1,398","1,378"


In [31]:
''' Daily : TOL Gross Adds Connected : Consumer '''

v_metict_cd = 'TB3S000102CS'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_tol_conn_con_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_tol_conn_con_df = issue_daily_tol_conn_con_df.loc[issue_daily_tol_conn_con_df['METRIC_CD']==v_metict_cd]
issue_daily_tol_conn_con_df = issue_daily_tol_conn_con_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_tol_conn_con_df_display = issue_daily_tol_conn_con_df.copy()
mod_col_list = issue_daily_tol_conn_con_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_tol_conn_con_df_display[col] = issue_daily_tol_conn_con_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_tol_conn_con_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202507,20250702,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,nan,nan,nan,nan
1,202507,20250701,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-07-03 03:13:24,"1,719","1,398","1,398","1,378"


## ** Current Issue